# Image Classification with CNN for Malaria Data

CNN With Custom Images. The dataset contains 2 folders - Infected - Uninfected. And a total of 27,558 images.

You can find the dataset [here](https://lhncbc.nlm.nih.gov/LHC-downloads/downloads.html#malaria-datasets)

In [1]:
import cv2
import pandas as pd
import os
import matplotlib.pyplot as plt

In [2]:
labels =['Parasitized','Uninfected']
img_path='cell_images/'

In [3]:
img_list=[]
label_list=[]
for label in labels:
    for img_file in os.listdir(img_path+label):
        img_list.append(img_path+label+'/'+img_file)
        label_list.append(label)

In [4]:
df=pd.DataFrame({'img':img_list,'label':label_list})

In [5]:
df.head()

,img,label
0,cell_images/Parasitized/C100P61ThinF_IMG_20150...,Parasitized
1,cell_images/Parasitized/C100P61ThinF_IMG_20150...,Parasitized
2,cell_images/Parasitized/C100P61ThinF_IMG_20150...,Parasitized
3,cell_images/Parasitized/C100P61ThinF_IMG_20150...,Parasitized
4,cell_images/Parasitized/C100P61ThinF_IMG_20150...,Parasitized


In [6]:
d={'Parasitized':1,'Uninfected':0}

In [7]:
df['encode_label']=df['label'].map(d)

In [8]:
df.head()

,img,label,encode_label
0,cell_images/Parasitized/C100P61ThinF_IMG_20150...,Parasitized,1
1,cell_images/Parasitized/C100P61ThinF_IMG_20150...,Parasitized,1
2,cell_images/Parasitized/C100P61ThinF_IMG_20150...,Parasitized,1
3,cell_images/Parasitized/C100P61ThinF_IMG_20150...,Parasitized,1
4,cell_images/Parasitized/C100P61ThinF_IMG_20150...,Parasitized,1


In [9]:
df.tail()

,img,label,encode_label
27553,cell_images/Uninfected/C99P60ThinF_IMG_2015091...,Uninfected,0
27554,cell_images/Uninfected/C99P60ThinF_IMG_2015091...,Uninfected,0
27555,cell_images/Uninfected/C99P60ThinF_IMG_2015091...,Uninfected,0
27556,cell_images/Uninfected/C99P60ThinF_IMG_2015091...,Uninfected,0
27557,cell_images/Uninfected/C99P60ThinF_IMG_2015091...,Uninfected,0


In [10]:
import numpy as np

In [11]:
x=[]
for img in df['img']:
    img=cv2.imread(img)
    img=cv2.resize(img,(170,170))  #boyutunu 170*170 pixel yaptık
    img=img/255.0  #Normalize ettik
    x.append(img)
x=np.array(x)

In [12]:
y=df['encode_label']

In [13]:
from sklearn.model_selection import train_test_split

In [14]:
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.20,random_state=42)

In [15]:
from keras.models import Sequential
from keras.layers import  Dense, Conv2D, Input, Reshape, MaxPooling2D, Flatten, Dropout, BatchNormalization

## CNN Convolutional Neural Network

In [16]:
model=Sequential()
model.add(Input(shape=(170,170,3)))
model.add(Conv2D(32,kernel_size=(3,3),activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.2))
model.add(Conv2D(64,kernel_size=(3,3),activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.2))
model.add(Flatten())
model.add(Dense(128))
model.add(Dense(1, activation='sigmoid')) 
model.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])

In [17]:
history=model.fit(x_train,y_train,validation_data=(x_test,y_test),epochs= 5,verbose=1)

Epoch 1/5
689/689 ━━━━━━━━━━━━━━━━━━━━ 285s 405ms/step - accuracy: 0.5734 - loss: 2.7048 - val_accuracy: 0.8208 - val_loss: 0.4398
Epoch 2/5
689/689 ━━━━━━━━━━━━━━━━━━━━ 262s 380ms/step - accuracy: 0.8181 - loss: 0.4583 - val_accuracy: 0.8627 - val_loss: 0.3575
Epoch 3/5
689/689 ━━━━━━━━━━━━━━━━━━━━ 255s 371ms/step - accuracy: 0.8874 - loss: 0.3365 - val_accuracy: 0.8991 - val_loss: 0.3326
Epoch 4/5
689/689 ━━━━━━━━━━━━━━━━━━━━ 237s 345ms/step - accuracy: 0.8852 - loss: 0.3368 - val_accuracy: 0.8590 - val_loss: 0.3818
Epoch 5/5
689/689 ━━━━━━━━━━━━━━━━━━━━ 235s 342ms/step - accuracy: 0.8827 - loss: 0.3354 - val_accuracy: 0.8794 - val_loss: 0.3378


In [18]:
model.save('my_malariia.h5')

In [19]:
# VGGNET, REsNet50,InceptionV3, Xception, MobileNetV2 Transfer Learning

In [20]:
from keras.models import Sequential
from keras.layers import Conv2D, Dense, Flatten, Input, MaxPooling2D, Dropout,BatchNormalization, Reshape
from tensorflow.keras.applications import VGG16, VGG19, ResNet50
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [21]:
data_dir='cell_images/'
img_Width,img_Height=224,224

train_datagen=ImageDataGenerator(rescale=1./255,validation_split=0.20)
train_datagenerator=train_datagen.flow_from_directory(directory=data_dir,target_size=(img_Width,img_Height),
                                 class_mode='binary',subset='training')
test_datagen=ImageDataGenerator(rescale=1./255)
test_datagenerator=train_datagen.flow_from_directory(directory=data_dir,target_size=(img_Width,img_Height),
                                 class_mode='binary',subset='validation')

base_model=ResNet50(weights='imagenet',input_shape=(img_Width,img_Height,3),include_top=False)

model=Sequential()
model.add(base_model)
for layer in base_model.layers:
    layer.trainable=False
model.add(Flatten())
model.add(Dense(1024,activation='relu'))
model.add(Dense(1,activation='sigmoid'))

model.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])

model.fit(train_datagenerator,epochs=1,validation_data=test_datagenerator)


Found 22048 images belonging to 2 classes.
Found 5510 images belonging to 2 classes.


C:\Users\elifs\AppData\Roaming\Python\Python311\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:122: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


689/689 ━━━━━━━━━━━━━━━━━━━━ 1771s 3s/step - accuracy: 0.5951 - loss: 1.7306 - val_accuracy: 0.6844 - val_loss: 0.6034


In [22]:
model.summary()

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ resnet50 (Functional)                │ (None, 7, 7, 2048)          │      23,587,712 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten_1 (Flatten)                  │ (None, 100352)              │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_2 (Dense)                      │ (None, 1024)                │     102,761,472 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_3 (Dense)                      │ (None, 1)                   │           1,025 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 331,875,205 (1.24 GB)

 Trainable params: 102,762,497 (392.01 MB)

 Non-trainable params: 23,587,712 (89.98 MB)

 Optimizer params: 205,524,996 (784.02 MB)

In [23]:
data_dir='cell_images/'
img_Width,img_Height=224,224

train_datagen=ImageDataGenerator(rescale=1./255,validation_split=0.20)
train_datagenerator=train_datagen.flow_from_directory(directory=data_dir,target_size=(img_Width,img_Height),
                                 class_mode='binary',subset='training')
test_datagen=ImageDataGenerator(rescale=1./255)
test_datagenerator=train_datagen.flow_from_directory(directory=data_dir,target_size=(img_Width,img_Height),
                                 class_mode='binary',subset='validation')

base_model=VGG16(weights='imagenet',input_shape=(img_Width,img_Height,3),include_top=False)

model=Sequential()
model.add(base_model)
for layer in base_model.layers:
    layer.trainable=False
model.add(Flatten())
model.add(Dense(1024,activation='relu'))
model.add(Dense(1,activation='sigmoid'))

model.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])

model.fit(train_datagenerator,epochs=1,validation_data=test_datagenerator)


Found 22048 images belonging to 2 classes.
Found 5510 images belonging to 2 classes.
689/689 ━━━━━━━━━━━━━━━━━━━━ 3804s 6s/step - accuracy: 0.8263 - loss: 0.7291 - val_accuracy: 0.9020 - val_loss: 0.2381


In [24]:
model.summary()

Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ vgg16 (Functional)                   │ (None, 7, 7, 512)           │      14,714,688 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten_2 (Flatten)                  │ (None, 25088)               │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_4 (Dense)                      │ (None, 1024)                │      25,691,136 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_5 (Dense)                      │ (None, 1)                   │           1,025 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 91,791,173 (350.16 MB)

 Trainable params: 25,692,161 (98.01 MB)

 Non-trainable params: 14,714,688 (56.13 MB)

 Optimizer params: 51,384,324 (196.02 MB)

In [25]:
data_dir='cell_images/'
img_Width,img_Height=224,224

train_datagen=ImageDataGenerator(rescale=1./255,validation_split=0.20)
train_datagenerator=train_datagen.flow_from_directory(directory=data_dir,target_size=(img_Width,img_Height),
                                 class_mode='binary',subset='training')
test_datagen=ImageDataGenerator(rescale=1./255)
test_datagenerator=train_datagen.flow_from_directory(directory=data_dir,target_size=(img_Width,img_Height),
                                 class_mode='binary',subset='validation')

base_model=VGG19(weights='imagenet',input_shape=(img_Width,img_Height,3),include_top=False)

model=Sequential()
model.add(base_model)
for layer in base_model.layers:
    layer.trainable=False
model.add(Flatten()) 
model.add(Dense(1024,activation='relu'))
model.add(Dense(1,activation='sigmoid'))

model.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])

model.fit(train_datagenerator,epochs=1,validation_data=test_datagenerator)


Found 22048 images belonging to 2 classes.
Found 5510 images belonging to 2 classes.
689/689 ━━━━━━━━━━━━━━━━━━━━ 4525s 7s/step - accuracy: 0.7873 - loss: 0.8611 - val_accuracy: 0.9013 - val_loss: 0.2637


In [26]:
model.summary()

Model: "sequential_3"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ vgg19 (Functional)                   │ (None, 7, 7, 512)           │      20,024,384 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten_3 (Flatten)                  │ (None, 25088)               │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_6 (Dense)                      │ (None, 1024)                │      25,691,136 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_7 (Dense)                      │ (None, 1)                   │           1,025 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 97,100,869 (370.41 MB)

 Trainable params: 25,692,161 (98.01 MB)

 Non-trainable params: 20,024,384 (76.39 MB)

 Optimizer params: 51,384,324 (196.02 MB)